# Getting the SVs close to the META 5 risk loci

Take the LD structure of the variant

In [1]:
import pandas as pd

In [2]:
meta5 = pd.read_csv('../../../GWAS_prog/data/Meta5new.csv')
meta5 = meta5[meta5['Failed final filtering and QC']==0]

In [3]:
dbSNPs = meta5.SNP
dbSNPs.to_csv('t/toextract.txt', index=False)

# Get the genotype of 90 SNPs

In [4]:
# meta5[['SNP', 'Effect allele']]
df = pd.DataFrame(data = {'SNP': meta5.SNP, 'A1': meta5['Effect allele'].str.upper()})
df[['SNP', 'A1']].to_csv('t/SNPs.txt', index=False, sep='\t')

In [7]:
%%bash
BFILE='../../../../PPMI_WGS/july_2018/PPMI_july2018'
plink --bfile $BFILE \
  --extract 't/SNPs.txt' \
  --recode A \
  --recode-allele 't/SNPs.txt' \
  --out result/risk
# grep 'G'$'\t''A'$'\t''rs35749011' $ANNOVAR_DATA/hg38/hg38_avsnp150.txt

PLINK v1.90b4.4 64-bit (21 May 2017)           www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to result/risk.log.
Options in effect:
  --bfile ../../../../PPMI_WGS/july_2018/PPMI_july2018
  --extract t/SNPs.txt
  --out result/risk
  --recode A
  --recode-allele t/SNPs.txt

257652 MB RAM detected; reserving 128826 MB for main workspace.
49900024 variants loaded from .bim file.
1379 people (777 males, 602 females) loaded from .fam.
1249 phenotype values loaded from .fam.
--extract: 90 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1379 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999903.
90 variants and 1379 people pass filter

In [9]:
%%bash 
rm result/risk.log

# Get the range of LD for each SNV
Using plink tag finder    
Need to liftover from GRCh38 to GRCh37

In [82]:
%%bash
mkdir LDwithM5

In [83]:
%%bash
BFILE='../../../../PPMI_WGS/july_2018/PPMI_july2018'
plink --bfile $BFILE \
  --recodeA
  --show-tags 't/toextract.txt' \
  --list-all \
  --tag-kb 500 \
  --tag-r2 0.8 \
  --out LDrisk_500W/plink
# grep 'G'$'\t''A'$'\t''rs35749011' $ANNOVAR_DATA/hg38/hg38_avsnp150.txt

PLINK v1.90b4.4 64-bit (21 May 2017)           www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to LDwithM5/plink.log.
Options in effect:
  --bfile ../../../../PPMI_WGS/july_2018/PPMI_july2018
  --list-all
  --out LDwithM5/plink
  --show-tags t/toextract.txt
  --tag-kb 500
  --tag-r2 0.8

257652 MB RAM detected; reserving 128826 MB for main workspace.
49900024 variants loaded from .bim file.
1379 people (777 males, 602 females) loaded from .fam.
1249 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1379 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.992562.
49900024 variants and 1379 people pass filters and QC.
Amon

In [18]:
ldrange = pd.read_csv('LDrisk/plink.tags.list', delim_whitespace=True)
t = 'chr' + ldrange['CHR'].apply(str) + ':' + ldrange['LEFT'].apply(str) + '-' + ldrange['RIGHT'].apply(str)
t.to_csv('LDrisk/hg38_range.txt', index=False)

liftover from with [genome browser online tool](https://genome.ucsc.edu/cgi-bin/hgLiftOver)

In [42]:
t = pd.read_csv('LDrisk/hglft_37.bed', header=None)
t[['chr', 'T']]= t[0].str.split(':', expand=True)
t[['start', 'end']] = t['T'].str.split('-', expand=True)
t['SNP'] = ldrange['SNP']
t['LENGTH'] = t['start'].apply(int) - t['end'].apply(int)
t[['SNP','chr', 'start', 'end', 'LENGTH']].to_csv('LDrisk/hg19_range.csv', index=False)

In [43]:
Rscript = """
library(data.table);library(dplyr)
# Paramaters
fnames = list.files("manta/")
fnames = fnames[grep("PPMISI", fnames)] # exclude some file names

# filename = "manta/PPMISI10874.diploidSV.vcf.gz"
# folder = tstrsplit(filename, "/") %>% .[[1]]
loci= fread('LDrisk/hg19_range.csv') %>% filter(complete.cases(.)) %>% 
  select(-LENGTH) %>% data.frame()

for (i in 1:length(fnames)){
  filename = paste("manta", fnames[i], sep = "/") 
  PATNO = tstrsplit(filename, "PPMISI") %>% .[[2]] %>% tstrsplit(., "\\\\.") %>% .[[1]]
  output = paste("t/", "M5_PPMISI", PATNO, sep = "")
  
  # Read diploid model outputs
  dip = fread(cmd = paste("zless", filename, "| grep -v '##'"))
  dip = dip %>% filter(FILTER=="PASS")
  dip = dip %>% filter(`#CHROM` %in% paste("chr", c(1:22, "X"), sep = "")) %>%
    rename(chr = `#CHROM`)
  
  # Filter SVs by their positions
  j = inner_join(dip, loci, by = "chr") %>% 
    mutate(PATNO = as.numeric(PATNO)) %>% 
    filter(POS> start - 1000 & POS < end + 1000) %>% # 1K margin
    fwrite(., paste(output, '1K.txt', sep = "_"))
}
"""
with open('script/deriveRiskLoci.R', 'w') as f:
    f.write(Rscript)

In [44]:
!Rscript --vanilla script/deriveRiskLoci.R # takes 30 min


Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [45]:
Rscript = """
library(data.table);library(dplyr)
KEY = "t/M5_*_1K.txt"
tempfunc = function(K){
  t = fread(cmd = paste("awk 'NR==1{print;next}FNR>1{print}'", K))
  names(t)[10]="GENOTYPE"
  dx = fread("../../../PDcohorts/PPMI/out181018/DEMOG_DIAG.csv")
  tab = semi_join(dx, t, by = "PATNO") %>% with(table(RECRUIT)) %>% data.frame(.) 
  j = inner_join(t, dx, by = "PATNO") %>% inner_join(., tab, by="RECRUIT")
  j %>% with(table(SNP, RECRUIT))
  dip = j %>% 
    tidyr::separate("GENOTYPE", c("GENOTYPE", rep(NA, 5)), sep=":") %>% 
    mutate(DOSE = case_when(
      GENOTYPE=="1/1" ~ 2, 
      GENOTYPE=="1/0" ~ 1,
      GENOTYPE=="0/1" ~ 1,
      GENOTYPE=="0/0" ~ 0,
      TRUE ~ 99))
  t = dip %>% select(ID) %>%
    tidyr::separate(ID, paste("V", 1:8, sep=""), sep=":", fill = 'right')
  dip$TYPE = ifelse(is.na(as.numeric(t$V2)), paste(t$V1, t$V2, sep = ":"), t$V1)
  
  # Get the SV length
  t = dip %>% select(INFO) %>%
    tidyr::separate(INFO, paste("V", 1:10, sep=""), sep=";", fill = 'right')
  getLEN = function(x){
    v = t[x,] %>% t %>% as.vector
    j = grep("SVLEN\\\\=", v)
    if(length(j)==1){
      g = v[j]
      SVLEN = tstrsplit(g, "\\\\=")[[2]][1] %>% as.numeric
    }else{SVLEN=NA}
    return(SVLEN)
  }
  dip$SVLEN = lapply(1:nrow(t), getLEN) %>% unlist
  out = sub("t/M5_\\\\*", "result/risk", KEY) %>% sub("txt", "csv", .) # Name for the output file
  fwrite(dip, out, row.names = F)
}
tempfunc(KEY)
"""
with open('script/populateRiskLoci.R', 'w') as f:
    f.write(Rscript)

In [46]:
!Rscript --vanilla script/populateRiskLoci.R


Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning messages:
1: Column `RECRUIT` joining character vector and factor, coercing into character vector 
2: Expected 6 pieces. Missing pieces filled with `NA` in 5046 rows [6, 11, 23, 33, 42, 46, 47, 49, 50, 56, 69, 73, 74, 87, 93, 109, 119, 126, 127, 128, ...]. 
3: In ifelse(is.na(as.numeric(t$V2)), paste(t$V1, t$V2, sep = ":"),  :
  NAs introduced by coercion


In [48]:
# !head result/risk_1K.csv

In [52]:
Rscript = """
library(data.table);library(dplyr)
# derived SVs
fname = "result/risk_1K.csv"
dip = fread(fname)

# risk doses
frisk = 'result/risk.raw'
risk = fread(frisk) %>% 
  mutate(PATNO = sub("PPMISI", "", FID) %>% as.numeric()) %>% 
  filter(PATNO %in% unique(dip$PATNO))


# Give SVID
SVIDtab = dip %>% distinct(chr, POS, REF, ALT, TYPE, SVLEN) %>% 
  mutate(SVID = as.numeric(rownames(.)))

# create evel df
df = inner_join(dip, SVIDtab, by =c("chr", "POS", "REF", "ALT", "TYPE", "SVLEN")) %>% 
  arrange(desc(QUAL)) %>% # if duplicated, take the higher QUAL
  distinct(PATNO, SNP, SVID, DOSE) 

# create spread df
SNPs = unique(dip$SNP)

iterate_snp = function(snp){
  dfs = df %>% filter(SNP == snp) %>% # extrct SVs close to a particular snp and create wide table
    tidyr::spread(key = SVID, value = DOSE)
  IDX = grep(paste(snp, "_", sep=""), names(risk)) # the col num for risk snp
  dfsc = full_join(dfs, risk[,c(IDX, ncol(risk))], by = "PATNO")
  dfsc[is.na(dfsc)]=0
# print(paste(snp, ncol(dfsc)-1))
  SVIDs = names(dfsc)[3:(ncol(dfsc)-1)]
  t = lapply(SVIDs, function(x) getLD(x, snp, dfsc)) %>% do.call(rbind, .) %>% data.frame()
  return(t)
}

getLD = function(svid, snp, dfsc){
  x = dfsc[,svid]
  y = dfsc[, ncol(dfsc)]
  res = lm(y~x) %>% summary
  res = c(snp, svid, res$r.squared)
  return(res)
}

t = lapply(SNPs, iterate_snp) %>% do.call(rbind, .) %>% data.frame() %>% 
  mutate(X2 = as.numeric(as.character(X2)),
         X3 = as.numeric(as.character(X3)))

names(t)=c("SNP_target", "SVID", "Rsq")

output = left_join(t, SVIDtab, by = 'SVID') %>% arrange(desc(Rsq))
fwrite(output, "result/risk_1K_Rsq.csv", row.names = F)
"""
with open('script/RiskSV_Rsq.R', 'w') as f:
    f.write(Rscript)

In [53]:
!Rscript --vanilla script/RiskSV_Rsq.R


Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [55]:
from IPython.display import FileLink
FileLink('result/risk_1K_Rsq.csv')

/gpfs/gsfs9/users/LNG/iwakih2/dataset/PPMI/SVs/result/risk_1K_Rsq.csv

In [49]:
!grep rs4140646 ../../../GWAS_prog/data/Meta5new.csv

rs4140646,6,27738801,LOC100131289,TXNDC15,a,g,0.2081,0.0833,0.0121,5.62E-12,5.81E-12,5.62E-12,5.66E-08,2.68E-08,0,0.2037,0.095,0.0145,5.88E-11,0,0.2181,0.0568,0.0219,9.61E-03,0,T,1,0,0,29
